# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd

import nltk
import pickle
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sqlalchemy import create_engine
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterMessagesDatabase.db')
df = pd.read_sql_table("DisasterMessagesTable", con=engine)


In [3]:
df.head(1)

,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df["genre"].value_counts()

news      13053
direct    10766
social     2395
Name: genre, dtype: int64

In [5]:
X = df["message"]
y = df.drop(columns=["message", "original", "genre"])

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
        
    tokens = word_tokenize(text,language='english',)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = [lemmatizer.lemmatize(tok).lower().strip() for tok in tokens]
    
    removed_stop_words = [tok for tok in clean_tokens if tok not in stopwords.words("english")]
    
    return removed_stop_words


In [7]:
X.iloc[10]

"There's nothing to eat and water, we starving and thirsty."

In [8]:
tokenize(X[10])

['nothing', 'eat', 'water', 'starving', 'thirsty']

In [9]:
y.iloc[10]

related                   1
request                   1
offer                     0
aid_related               1
medical_help              1
medical_products          1
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     1
food                      1
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 1
infrastructure_related    1
transport                 1
buildings                 1
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      1
weather_related           1
floods                    1
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report       

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
y_test

In [ ]:
y_test.shape

In [ ]:
y_pred

In [ ]:
y_pred.shape

In [ ]:
y_test.iloc[0]

In [ ]:
y_test.iloc[:,0].unique()

In [ ]:
for i in range(len(y_train.columns)):
    print(f"For {y_train.columns[i]}:")
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))
    print("-"*50)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params()

In [ ]:
parameters = {
    'clf__estimator__max_depth': [5, 10, 25, 50, None],
    'clf__estimator__min_samples_leaf': [1,2,5]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2)

cv.fit(X_train, y_train)

In [ ]:
cv.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = cv.predict(X_test)

for i in range(len(y_train.columns)):
    print(f"For {y_train.columns[i]}:")
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))
    print("-"*50)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

Tried to use Linear SVC (as per https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) but was unable to get it working in the time limit

### 9. Export your model as a pickle file

In [ ]:
best_model = cv.best_estimator_

with open("best_model.pickle", "w") as f:
    pickle.dump(best_model, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.